# Data Pull

#### Imports

In [3]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
# from html_table_parser.parser import HTMLTableParser
from bs4 import BeautifulSoup
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
# import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice


#### Quandl/WRDS Credentials

Importing quandl API key; here, I protect it in a ```.json``` file.

In [4]:
with open('quandl_config.json', 'r') as f:
    config = json.load(f)
    quandl_api_key = config['api_key']

quandl.ApiConfig.api_key = quandl_api_key

In [5]:
with open('wrds_config.json', 'r') as config_file:
    wrds_credentials = json.load(config_file)

**Instructions for local use**

The API keys and WRDS login are loaded from *quandl_config.json* in the same directory as this notebook (local), which has the following content,

```
{
    "api_key": ""
}
```

and *wrds_config.json*,

```
{
    "wrds": {
        "username": "",
        "password": ""
    }
}
```

You will need to set up a file ```.pgpass``` with,
```
hostname:port:database:username:password
wrds-pgdata.wharton.upenn.edu:9737:wrds:(user):(pass)
```
if you would like to skip having to input your username and password every time.

# Data Load

### Quandl

Compatibility: ```os.environ["HOME"]``` to ```os.getcwd()``` and changing ```os.mkdir``` to ```os.makedirs```.

In [6]:
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(os.getcwd(), "quandl_data_table_downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.makedirs(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename=data_file, api_key=quandl_api_key, **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"


In [7]:
def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

## Pull SPY Data

In [11]:
datapull = data_pull = fetch_quandl_table('QUOTEMEDIA/PRICES', ticker='SPY', avoid_download=True)

Download finished: 405410 bytes
Creating symlink: C:\Users\axrpo\Documents\GitHub\qts_final\quandl_data_table_downloads\QUOTEMEDIA/PRICES_20240220.zip -> C:\Users\axrpo\Documents\GitHub\qts_final\quandl_data_table_downloads\QUOTEMEDIA/PRICES_latest.zip


In [12]:
datapull.to_csv('spy_tickerdata.csv', index=False)

In [13]:
datapull

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,SPY,1993-08-13,45.093700,45.156200,45.093700,45.125000,103500.0,0.0,1.0,25.778428,25.814157,25.778428,25.796321,103500.0
1,SPY,1993-08-16,45.156200,45.500000,45.156200,45.375000,241800.0,0.0,1.0,25.814157,26.010695,25.814157,25.939237,241800.0
2,SPY,1993-08-17,45.343700,45.531200,45.343700,45.531200,369300.0,0.0,1.0,25.921344,26.028531,25.921344,26.028531,369300.0
3,SPY,1993-08-18,45.687500,45.875000,45.656200,45.781200,414300.0,0.0,1.0,26.117882,26.225069,26.099989,26.171447,414300.0
4,SPY,1993-08-19,45.812500,45.812500,45.718700,45.781200,28500.0,0.0,1.0,26.189340,26.189340,26.135718,26.171447,28500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7814,SPY,1993-06-22,44.656200,44.656200,44.562500,44.625000,137500.0,0.0,1.0,25.528325,25.528325,25.474761,25.510490,137500.0
7815,SPY,1994-11-16,46.765598,46.843700,46.609299,46.843700,106900.0,0.0,1.0,27.589587,27.635663,27.497377,27.635663,106900.0
7816,SPY,2010-03-10,114.510002,115.279999,114.410004,114.970001,186088800.0,0.0,1.0,87.964535,88.556033,87.887719,88.317898,186088800.0
7817,SPY,2022-02-15,443.730000,446.280000,443.180000,446.100000,88435136.0,0.0,1.0,430.043935,432.515285,429.510899,432.340837,88435136.0


## Pull WRDS Options Data for SPY

### Connection Toggle

In [14]:
db = wrds.Connection(
    wrds_username=wrds_credentials['wrds']['username'],
)

Loading library list...
Done


In [15]:
db.close()

### Data Pull

Pulls the data and stores in .csv file in the same directory, so on the next init you don't have to run this.

In [15]:
start_year = 2018
end_year = 2023

In [16]:
optiondata = pd.DataFrame()

for year in range(start_year, end_year + 1):
    sql_query = f"""
    SELECT date, exdate, last_date, cp_flag, strike_price, best_bid, best_offer,
           volume, open_interest, impl_volatility, delta, gamma, vega, theta,
           expiry_indicator
    FROM optionm_all.opprcd{year}
    WHERE secid = '109820' 
    AND date BETWEEN '{year}-01-01' AND '2023-02-28'
    AND exdate - date < 800
    """
    year_data = db.raw_sql(sql_query)
    optiondata = pd.concat([optiondata, year_data], ignore_index=True)

Add AND volume >= 100 to filter volume.

In [17]:
csv_file_path = 'option_data.csv'
optiondata.to_csv(csv_file_path, index=False)
print(f"All data saved to {csv_file_path}")

All data saved to option_data.csv


### Retrieve as Variable

In [18]:
csv_file_path = 'option_data.csv'
spydata = pd.read_csv(csv_file_path)

In [19]:
display(spydata)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator
0,2018-01-02,2018-01-03,2017-12-28,C,235000.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w
1,2018-01-02,2018-01-03,2018-01-02,C,240000.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w
2,2018-01-02,2018-01-03,2017-12-27,C,242500.0,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w
3,2018-01-02,2018-01-03,2018-01-02,C,245000.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w
4,2018-01-02,2018-01-03,NaN,C,247500.0,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600000.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448190,2023-02-28,2025-03-21,NaN,P,605000.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448191,2023-02-28,2025-03-21,NaN,P,610000.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448192,2023-02-28,2025-03-21,NaN,P,615000.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
